# Extended interactions in 1D and charge density waves
Consider an extended Hubbard model in dimension 1 with a local interaction $U$, and first-neighbor hopping term $-t$ and a first-neighbor interaction $V$.
Construct a 4-site cluster to represent this system, with a Hartree mean field $V_m$ designed to represent the effect of the inter-cluster part of the extended interaction (see documentation on the Hartree approximation).
Construct also an operator $\hat\Delta$ reprensenting a site-centered charge density wave of period 2:
$$ \hat\Delta = \sum_i (-1)^i n_i  $$
In this problem we will stay at half-filling, where the chemical potential can be shown to be $\mu=U/2+2V$. We will set $U=4$ and $t=1$.

- This part is purely theoretical. Show that, in the $t\to 0$ limit, the system develops a spontaneous charge density wave if $V>V_c = U/2$.
Does this critical value of $V$ change if $t$ is no longer zero? If so, in what direction?

- In the absence of charge density wave, find the optimal value of the Hartree field $V_m$ as a function of $V$ in the range $V\in[0,U]$.
For this, proceed by using the self-consistency condition and the function $\texttt{pyqcm.loop.Hartree()}$.

- Redo the above, this time using the VCA with the coefficient of $V_m$ as a variational parameter (this is a lattice, not a cluster parameter).
Beware of the keyword argument $\texttt{hartree}$ in the function $\texttt{vca()}$; it is crucial.

- Using as an \textit{additional} variational parameter the coefficient of $\hat\Delta$ (this is a cluster, not a lattice parameter), find how the charge density wave develops as a function of $V$ in the interval $V\in[3,4]$. It is easier for this to loop down from $V=4$ and to start with the
appropriate value of $V_m$ as a starting value. A quick $\texttt{plot\_sef()}$ may be useful here to find an appropriate starting value of $\Delta$.
Make a plot of the order parameter $\langle\hat\Delta\rangle$ as a function of $V$. You should find here a confirmation to your answer to the last question raised in part A.


### Spontaneous charge density formation

In the $t\to0$ limit, the hamiltonian can be written as:
$$
    H = U + V - \mu\hat N = U\sum_i n_{i\uparrow}n_{i\downarrow} + V\sum_i n_{i}n_{i+1} - \mu\sum_i n_{i}
$$
Consider a charge density wave configuration at half-filling with site occupations
$$ n_i = 1 + (-1)^i $$
Every other site, there is a contribution of $U$ and $-2\mu$ to the energy but there is obviously no first-neighbor interaction terms contributing. The energy per site is therefore: 
$$\epsilon_{\text{cdw}} = \frac{U}{2} - \mu$$
In the case of a uniform distribution of electrons at half-filling, the site occupation is trivially $n_i=1$.
In this case there is a contribution of $V$ and $-\mu$ every site but not from on-site interactions. Therefore
$$ \epsilon_{\text{normal}} = V - \mu$$
For a spontaneous CDW to develop, the energy of a uniform system must be greater than that of the charge density wave created. This condition is simply stated as:
\begin{align*}
    \epsilon_{\text{normal}} &> \epsilon_{\text{cdw}} \\
    V_c - \mu &> \frac{U}{2} - \mu\\
    V_c &> \frac{U}{2}
\end{align*}
The critical value $V_c$ for which a charge density wave can spontaneously arise is $V_c = U/2$. Of course, if $t$ is not trivial, $V_c$ will increase since kinetic energy has a tendency to spread out the electrons more evenly, thus requiring a stronger $V$ to overcome this effect.

In [ ]:
from pyqcm import *
from pyqcm.draw_operator import *
from pyqcm.hartree import *
from pyqcm.loop import *
from pyqcm.vca import *
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
# A simple 1D 4 site cluster
new_cluster_model("clus", 4, 0)

add_cluster("clus", [0,0,0], [[0,0,0],[1,0,0],[2,0,0],[3,0,0]])
lattice_model("lat", [[4,0,0]])

### A note on $V_m$

Since the sites on either end of the cluster are the only ones that will feel the effect of inter-cluster interactions the components of $V^m_{ij}$ where $i$ and $j$ are site indices are:
$$
    V^m_{ij} \propto
    \begin{pmatrix}
        0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        1 & 0 & 0 & 0
    \end{pmatrix}
$$
This matrix admits $1$, $-1$ and $0$ twice as eigenvalues, but only the first two ones are physically interesting.

In [ ]:
interaction_operator("U", amplitude=1) # on site interaction
interaction_operator("V", link=[1,0,0], amplitude=1) # NN interaction

# Defining an explicit operator for the "Hartree part" of the extended interaction
elems = [([0,0,0], [0,0,0], 1/np.sqrt(2)), ([3,0,0], [0,0,0], 1/np.sqrt(2))] # the 1/sqrt(2) factor ensures normalization
explicit_operator("Vm", elems, type="one-body", tau=0) # Vm is an on-site operator ---> tau=0

hopping_operator("t", [1,0,0], -1) 

density_wave("Delta", "N", [1,0,0], amplitude=1) # A charge density wave with period 2 in position space

#print_model("model.out")    

In [ ]:
# Parameters required for half-filling
set_target_sectors(["R0:N4:S0"])
set_parameters("""
    U=4
    V=4
    Vm=5
    mu=10
    t=1
    Delta_1=1e-9
    Delta=1e-9
""")
I = new_model_instance()

In [ ]:
# Updates mu as a function of V and U
def update_mu():
    new_model_instance()

    V = parameters()["V"]
    U = parameters()["U"]
    set_parameter("mu", 2*V + U/2) # Condition to impose half-filling
    
    new_model_instance()

In [ ]:
# Defining an object of the hartree class with eigenvalue 1
Vm_obj = hartree("Vm", "V", 1, lattice=True) # lattice=True ---> use lattice averages

In [ ]:
#########################################################################################
############################## - Range for sweep over V - ###############################

V_start = 4
V_stop = 0
V_step = -0.1

#########################################################################################
############################# - Self-consistency approach - #############################

# This is simply a loop that applies the self_consistency approach over V_range
for V in np.arange(V_start, V_stop, V_step):
    set_parameter("V", V)
    new_model_instance()
    update_mu()

    Hartree(new_model_instance, [Vm_obj])

#########################################################################################
################################### - VCA approach - ####################################

# This is the function to run inside of controlled_loop to perform the vca itself
def run_vca():
    update_mu()
    vca(names=["Vm"], steps=[0.05], accur=[2e-3], max=[20], hartree=[Vm_obj], max_iter=300)    


set_parameter("Vm", 5.65) # Initial guess for the VCA based on the Hartee self-consistency
new_model_instance()
update_mu()

# performing the vca loop over V_range
controlled_loop(run_vca, varia=["Vm"], loop_param="V", loop_range=(V_start, V_stop, V_step))

#########################################################################################
################################### - Reading data - ####################################

V_grid_size = int(np.abs(V_stop - V_start//V_step) + 1)

d_vca = np.genfromtxt("./vca.tsv", usecols=[6, 7]) # getting V and Vm
d_vca = d_vca[-V_grid_size:] # reading the relevant vca results

d_hartree = np.genfromtxt("./hartree.tsv", usecols=[5, 6]) # getting V and Vm
d_hartree = d_hartree[-V_grid_size:] # reading the relevant vca results

### Comparing the self-consistency approach to the VCA approach

As can be seen in the plot below, both approaches for calculating $V_m$ as a function of $V$ yield identical results.

In [ ]:
# Plotting V_m as a function of for both approaches
fig, ax = plt.subplots()

ax.plot(d_hartree[:,0], d_hartree[:,1], 'o', color="red", markersize=4, label="Self consistency")
ax.plot(d_vca[:,0], d_vca[:,1], 'o', color="blue", markersize=3, label="VCA")

ax.set_xlabel("$V$")
ax.set_ylabel("$V_m$")

ax.legend()

fig.show()

In [ ]:
# Plotting the Potthoff functionnal as a function of Delta to find a starting value
set_parameter("V", 4)
set_parameter("Vm", 5.6) # Based on previous results
new_model_instance()
update_mu()

Delta_grid = np.linspace(-2, 2, 200)

plot_sef("Delta_1", Delta_grid, hartree=[Vm_obj]) 

In [ ]:
#########################################################################################
################################## - V range control - ##################################

# Decide whether or not to cover up to V=5 also (BOOLEAN)
LOOP_UP = False

#########################################################################################
############################## - Range for sweep over V - ###############################

# Starting at a know solution and going down
V_start = 4
V_stop = 3
V_step = -0.02

# Starting at a know solution and going up
V_start_2 = 4
V_stop_2 = 5
V_step_2 = 0.02

#########################################################################################
############################# - VCA over both parameters - ##############################

# Function to perform the VCA
def run_vca_2():
    update_mu()
    vca(names=["Vm", "Delta_1"], steps=[0.05, 0.01], accur=[2e-3, 2e-3], max=[100, 100], hartree=[Vm_obj], max_iter=500) # VCA over both parameters    


set_parameter("V", 4)
set_parameter("Vm", 5.65)
set_parameter("Delta_1", 0.1)
new_model_instance()
update_mu()

# Looping from 4 to 3
controlled_loop(run_vca_2, varia=["Vm", "Delta_1"], loop_param="V", loop_range=(V_start, V_stop, V_step))

if LOOP_UP is True:
    set_parameter("V", 4)
    set_parameter("Vm", 5.65)
    set_parameter("Delta_1", 0.1)
    new_model_instance()
    update_mu()

    # Looping from 4 to 5
    controlled_loop(run_vca_2, varia=["Vm", "Delta_1"], loop_param="V", loop_range=(V_start_2, V_stop_2, V_step_2))

#########################################################################################
######################### - Hybrid VCA with self-consistency - ##########################

def run_vca_3():
    update_mu()
    vca(names=["Delta_1"], steps=[0.01], accur=[2e-3], max=[100], hartree=[Vm_obj], max_iter=500, hartree_self_consistent=True) 


set_parameter("V", 4)
set_parameter("Vm", 5.65)
set_parameter("Delta_1", 0.1)
new_model_instance()
update_mu()

# Looping from 4 to 3
controlled_loop(run_vca_3, varia=["Delta_1"], loop_param="V", loop_range=(V_start, V_stop, V_step))

if LOOP_UP is True:
    set_parameter("V", 4)
    set_parameter("Vm", 5.65)
    set_parameter("Delta_1", 0.1)
    new_model_instance()
    update_mu()

    # Looping from 4 to 5
    controlled_loop(run_vca_3, varia=["Delta_1"], loop_param="V", loop_range=(V_start_2, V_stop_2, V_step_2))

#########################################################################################
################################### - Reading data - ####################################

V_grid_size = int(np.abs((V_stop - V_start)//V_step) + 1) # Number of grid points from 4 to 3
V_grid_size_2 = int(np.abs((V_stop_2 - V_start_2)//V_step_2) + 1) # Number of grid points from 4 to 5

raw_vca_data = np.genfromtxt("./vca.tsv", delimiter="\t", usecols=[6, 4]) # getting V and Delta

# Selecting data according to whether the range over V is [3,4] or [3,5]
if LOOP_UP is True:
    d_varia = raw_vca_data[-2*V_grid_size-2*V_grid_size_2:-V_grid_size-V_grid_size_2] # getting 2 param VCA data from V = [3,5]
    d_self_consis = raw_vca_data[-V_grid_size-V_grid_size_2:] # getting hybrid VCA data from V = [3,5]
else:
    d_varia = raw_vca_data[-2*V_grid_size:-V_grid_size] # getting 2 param VCA data from V = [3,4]
    d_self_consis = raw_vca_data[-V_grid_size:] # getting hybrid VCA data from V = [3,4]

### Comparing the hybrid Hartree self-consistent VCA to the two parameter VCA

In this case, the results are very similar, but not as close as for the previous comparison.

In [ ]:
# Plotting both VCA approaches
fig, ax = plt.subplots()

ax.plot(d_self_consis[:,0], d_self_consis[:,1], 'o', color="red", markersize=4, label="Self consistency")
ax.plot(d_varia[:,0], d_varia[:,1], 'o', color="blue", markersize=3, label="VCA")

ax.set_xlabel("$V$")
ax.set_ylabel("$\langle\Delta\\rangle$")

ax.legend()

fig.show()